In [1]:
import os
os.chdir('D:\\Gnan\\DA\\KMU\\Prediction-COVID-19')
from utils import *
from model import *
from trainer import *

from tqdm.notebook import tqdm
from torch.optim.adam import Adam
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import warnings
warnings.filterwarnings('ignore')

In [2]:
path = 'Data/Variants/Infected/omicron/*.csv'
dic_files = Load_files.load_files(path, -14, -12)
data = dic_files['대구']
data

,stdDay,incDec,22D_variants,23A_variants
0,2022-12-19,973.0,467.04,9.73
1,2022-12-20,3625.0,1740.00,36.25
2,2022-12-21,3616.0,1735.68,36.16
3,2022-12-22,3197.0,1534.56,31.97
4,2022-12-23,2657.0,1275.36,26.57
...,...,...,...,...
132,2023-04-30,396.0,75.24,253.44
133,2023-05-01,122.0,23.18,78.08
134,2023-05-02,480.0,91.20,307.20
135,2023-05-03,464.0,88.16,296.96


In [3]:
learning_rates_list = [1e-3, 1e-4, 1e-5]
patiences_list = [20, 50]
num_layers_list = [1, 2, 4, 8]
batch_sizes_list = [32, 64]
hidden_sizes_list = [8, 16, 32]
dropout_list = [0.25]

hyperparameter_dict = {}
i = 0

for lr in learning_rates_list:
    for patience in patiences_list:
        for num_layers in num_layers_list:
            for batch_sizes in batch_sizes_list:
                for hidden_size in hidden_sizes_list:    
                    for dropout in dropout_list:
                        hyperparameter_dict[i] = [lr, patience, num_layers, batch_sizes, hidden_size, dropout]
                        i += 1
                        
print(i)

144


In [4]:
criterion = nn.MSELoss()

input_size = 3
sequence_length = 60
num_epochs = 10000

df = Prepare_df.processing(data, 'stdDay', '23A_variants')

x = df.iloc[:, 0:]
y = df.iloc[:,:1]

ms = MinMaxScaler()
ss = StandardScaler()

ss.fit(x)
ms.fit(y)

MinMaxScaler()

In [5]:
best_model = {}
models_list = ['RNN', 'LSTM', 'GRU', 'BiRNN', 'BiLSTM', 'BiGRU']

for num_model in range(6):
    rmse_min = np.inf
    for hyperparameter in tqdm(hyperparameter_dict.values(), desc = f'대구_{models_list[num_model]}', position = 0):
        
        lr = hyperparameter[0]
        patience = hyperparameter[1]
        num_layers = hyperparameter[2]
        batch_size = hyperparameter[3]
        hidden_size = hyperparameter[4]
        dropout = hyperparameter[5]
        #print('-------------------------------------------------------------------------------------')
        #print('lr:', lr, ' patience:', patience, ' num_layers:', num_layers, 
        #      ' batch_size:', batch_size, ' hidden_size:', hidden_size, ' dropout:', dropout)
    
        x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 68, 60, 1, batch_size, 'mto')
        
        if num_model == 0:
            model = RNN(input_size = input_size,
                        hidden_size = hidden_size,
                        sequence_length = sequence_length,
                        num_layers = num_layers, 
                        dropout = dropout, 
                        device = device).to(device)
            
        elif num_model == 1:
            model = LSTM(input_size = input_size,
                         hidden_size = hidden_size,
                         sequence_length = sequence_length,
                         num_layers = num_layers, 
                         dropout = dropout, 
                         device = device).to(device)
        
        elif num_model == 2:
            model = GRU(input_size = input_size,
                        hidden_size = hidden_size,
                        sequence_length = sequence_length,
                        num_layers = num_layers, 
                        dropout = dropout, 
                        device = device).to(device)
            
        elif num_model == 3:
            model = BiRNN(input_size = input_size,
                          hidden_size = hidden_size,
                          sequence_length = sequence_length,
                          num_layers = num_layers, 
                          dropout = dropout, 
                          device = device).to(device)

        elif num_model == 4:
            model = BiLSTM(input_size = input_size,
                           hidden_size = hidden_size,
                           sequence_length = sequence_length,
                           num_layers = num_layers, 
                           dropout = dropout, 
                           device = device).to(device)

        elif num_model == 5:
            model = BiGRU(input_size = input_size,
                          hidden_size = hidden_size,
                          sequence_length = sequence_length,
                          num_layers = num_layers, 
                          dropout = dropout, 
                          device = device).to(device)

        optimizer = Adam(model.parameters(), lr = lr)
    
        loss_list, model, epoch = Trainer.Many_to_One(train_loader, 
                                                      test_loader, 
                                                      model, 
                                                      criterion, 
                                                      optimizer, 
                                                      num_epochs, 
                                                      patience,
                                                      device)
    
        label_y, predicted = predict_mto(model, df, x_ss, y_ms)
    
        if rmse_min > rmse(label_y[-7:], predicted[-7:]):
            rmse_min = rmse(label_y[-7:], predicted[-7:])
            best_model[models_list[num_model]] = [rmse_min, model, loss_list, epoch, lr, patience, 
                                                  num_layers, batch_size, hidden_size, dropout]

대구_RNN:   0%|          | 0/144 [00:00<?, ?it/s]

대구_LSTM:   0%|          | 0/144 [00:00<?, ?it/s]

대구_GRU:   0%|          | 0/144 [00:00<?, ?it/s]

대구_BiRNN:   0%|          | 0/144 [00:00<?, ?it/s]

대구_BiLSTM:   0%|          | 0/144 [00:00<?, ?it/s]

대구_BiGRU:   0%|          | 0/144 [00:00<?, ?it/s]

In [6]:
best_model

{'RNN': [37.93866156067745,
  RNN(
    (rnn): RNN(3, 16, num_layers=4, batch_first=True, dropout=0.25)
    (fc1): Linear(in_features=960, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=64, bias=True)
    (fc3): Linear(in_features=64, out_features=1, bias=True)
    (relu): ReLU()
  ),
  [0.25554897636175156,
   0.20494659865895906,
   0.17348697409033775,
   0.144106679285566,
   0.11053623942037423,
   0.08600111864507198,
   0.06399410963058472,
   0.05437874048948288,
   0.04761094351609548,
   0.048286790028214455,
   0.047543382893006005,
   0.04797618091106415,
   0.04979746540387472,
   0.049494327356417976,
   0.04840566093722979,
   0.04496516908208529,
   0.04337216913700104,
   0.0396622009575367,
   0.03974110633134842,
   0.03959242875377337,
   0.038124505430459976,
   0.0375006931523482,
   0.033813660964369774,
   0.03133371099829674,
   0.029092008868853252,
   0.029136531054973602,
   0.027439994737505913,
   0.027770689378182094,
   0.024

In [9]:
for key, values in best_model.items():
    save_model(values[1].state_dict(), f"model/Infected/23A/{key}.pth")
    save_hyperparameter(values[3:], f"hyperparameter/Infected/23A/{key}.pkl")